In [1]:
from PIL import Image
from create_dataset import create_sub_masks, create_sub_mask_annotation

import matplotlib.pyplot as plt
import os
import numpy as np
import tqdm
import datetime
import json
%matplotlib inline

category2id = {'car':33, 
               'person':36, 
               'truck':38,}
id2category = {}
for cat, _id in category2id.items():
    id2category[_id] = cat
    
label_name = os.listdir('./Dataset/sample_train_label/')
label_path = ['./Dataset/sample_train_label/'+name for name in label_name]
label_path.sort()
############# To Delete ###############
label_path = label_path[0:10]
############# To Delete ###############

img_name = os.listdir('./Dataset/sample_train_color/')
img_path = ['./Dataset/sample_train_label/'+name for name in img_name]
img_path.sort()
############# To Delete ###############
img_path = img_path[0:10]
############# To Delete ###############

In [30]:
print(img_path)

['./Dataset/170927_070141273_Camera_5.jpg', './Dataset/sample_train_label/171206_025742296_Camera_6.jpg', './Dataset/sample_train_label/171206_025742517_Camera_6.jpg', './Dataset/sample_train_label/171206_025742738_Camera_6.jpg', './Dataset/sample_train_label/171206_025742959_Camera_6.jpg', './Dataset/sample_train_label/171206_025743180_Camera_6.jpg', './Dataset/sample_train_label/171206_025743401_Camera_6.jpg', './Dataset/sample_train_label/171206_025743602_Camera_6.jpg', './Dataset/sample_train_label/171206_025743803_Camera_6.jpg', './Dataset/sample_train_label/171206_025744004_Camera_6.jpg']


# Create images list

In [27]:
def create_image_list(img_path):
    image_list = []
    for img_id, path in enumerate(img_path):
        image_list += [{
            'id': img_id,
            'license': 0,
            'coco_url': 'https://www.google.com/',
            'flickr_url': 'https://www.google.com/',
            'width': 3384,
            'height': 2710,
            'file_name': path.split('/')[-1],
            'date_captured': datetime.datetime.now().replace(microsecond=0).isoformat(' '),
        }]
    return image_list

image_list = create_image_list(img_path)

# Create annotations list

In [25]:
def create_object2color(label_path, id2category):
    
    class_instance = {}
    class_summary = {}
    for _id in id2category.keys():
        class_summary[_id] = set()
    
    for img_id, path in tqdm.tqdm(enumerate(label_path)):
        label = Image.open(path)
        label = np.array(label)
        label_class = np.ndarray.astype(label/1000, np.int32)
        label_instance = np.mod(label, 1000)
        
        class_instance[img_id] = []
        for class_id in id2category.keys():
            i, j = np.where(label_class==class_id)
            instance_set = set(label_instance[i, j])
            
            class_summary[class_id] = class_summary[class_id].union(instance_set)
            for instance_id in instance_set:
                class_instance[img_id] += [(class_id, instance_id)]
            
    object2color = {}
    color_set = set()
    def generate_color(color_set):
        while True:
            r, g, b = np.random.randint(256, size=3)
            if not (r, g, b) in color_set:
                break
        return (r, g, b)
    
    for class_id, instances_id in class_summary.items():
        for instance_id in instances_id:
            r, g, b = generate_color(color_set)
            object2color[(class_id, instance_id)] = (r, g, b)
            
    
    return object2color, class_instance

def do_mask_image(label, class_instance, object2color):
    label_class = np.ndarray.astype(label/1000, np.int32)
    label_instance = np.mod(label, 1000)
    
    h, w = label.shape
    image_mask = np.zeros([h, w, 3], np.int32)
    for class_id, instance_id in class_instance:
        mask = np.zeros([h, w, 3], np.int32)
        r, g, b = object2color[(class_id, instance_id)]
        
        intersect = (label_class==class_id)*(label_instance==instance_id)
        mask[:, :, 0] = r*intersect
        mask[:, :, 1] = g*intersect
        mask[:, :, 2] = b*intersect
        
        image_mask += mask
        
    return image_mask

def create_color2category(class_instance, object2color):
    category_ids = {}

    for img_id, pair_list in class_instance.items():
        category_ids[img_id] = {}
        for class_id, instance_id in pair_list:
            r, g, b = object2color[(class_id, instance_id)]
            category_ids[img_id][str((r, g, b))] = class_id
    
    return category_ids

In [4]:
object2color, class_instance = create_object2color(label_path, id2category)
category_ids = create_color2category(class_instance, object2color)

10it [00:04,  2.23it/s]


In [10]:
is_crowd = 0

# These ids will be automatically increased as we go
annotation_id = 0

# Create the annotations
annotations = []
for img_id, path in tqdm.tqdm(enumerate(label_path)):
    label = Image.open(path)
    label = np.array(label)
    mask = do_mask_image(label, class_instance[img_id], object2color)
    mask = Image.fromarray(np.uint8(mask))
    sub_masks = create_sub_masks(mask)
    for color, sub_mask in sub_masks.items():
        category_id = category_ids[img_id][color]
        annotation = create_sub_mask_annotation(sub_mask, img_id, category_id, annotation_id, is_crowd)
        annotations.append(annotation)
        annotation_id += 1
    
print(json.dumps(annotations))

10it [02:03, 10.85s/it]


[{"segmentation": [[48.0, 1873.5, 64.0, 1869.5, 68.0, 1866.5, 71.5, 1863.0, 79.0, 1848.5, 198.0, 1839.5, 199.5, 1848.0, 204.0, 1853.5, 223.0, 1859.5, 232.0, 1859.5, 239.0, 1857.5, 244.5, 1854.0, 254.0, 1836.5, 260.0, 1835.5, 261.0, 1830.5, 278.0, 1822.5, 280.5, 1815.0, 286.5, 1812.0, 281.5, 1781.0, 279.5, 1780.0, 278.5, 1775.0, 279.5, 1762.0, 253.5, 1732.0, 246.5, 1722.0, 246.5, 1720.0, 240.0, 1717.5, 162.0, 1713.5, 92.0, 1714.5, 33.0, 1722.5, 26.0, 1725.5, -0.5, 1740.0, -0.5, 1777.0, 10.5, 1782.0, 25.5, 1805.0, 25.5, 1838.0, 22.5, 1849.0, 16.5, 1856.0, 19.5, 1859.0, 22.5, 1866.0, 28.0, 1870.5, 48.0, 1873.5]], "iscrowd": 0, "image_id": 0, "category_id": 33, "id": 0, "bbox": [-0.5, 1713.5, 287.0, 160.0], "area": 34778.25}, {"segmentation": [[2.0, 1863.5, 16.0, 1858.5, 22.5, 1849.0, 25.5, 1838.0, 25.5, 1805.0, 10.5, 1782.0, 0.0, 1777.5, -0.5, 1863.0, 2.0, 1863.5]], "iscrowd": 0, "image_id": 0, "category_id": 33, "id": 1, "bbox": [-0.5, 1777.5, 26.0, 86.0], "area": 1792.0}, {"segmentation

In [29]:
data = {}
data['info'] = {}
data['licenses'] = []
data['images'] = image_list
data['annotations'] = annotations
data['categories'] = [
    {"supercategory": "vehicle", "id": 33,"name": "car"},
    {"supercategory": "vehicle", "id": 34,"name": "motorbicycle"},
    {"supercategory": "vehicle", "id": 35,"name": "bicycle"},
    {"supercategory": "vehicle", "id": 38,"name": "truck"},
    {"supercategory": "vehicle", "id": 39,"name": "bus"},
    {"supercategory": "vehicle", "id": 40,"name": "tricycle"},
    
    {"supercategory": "person", "id": 36,"name": "person"},
]
with open('./Dataset/data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [6]:
img = Image.open('./Dataset/170927_070141273_Camera_5.jpg')
label = Image.open('./Dataset/170927_070141273_Camera_5_instanceIds.png')
#label = Image.open(label_path[1])
label = np.array(label)
label_class = np.ndarray.astype(label/1000, np.int32)
label_instance = np.mod(label, 1000)

image_mask = do_mask_image(label, class_instance[0], object2color)

100%|███████████████████████████████████████████████████████████████████████████████| 51/51 [00:12<00:00,  4.31it/s]
